# 04 · Arquitectura del sistema de aprendizaje federado

En este notebook el objetivo es definir la arquitectura que vamos a usar para el aprendizaje federado (FL) sobre el problema de: **Predicción de muerte prematura por Enfermedades No Transmisibles (ENT / NCD)** usando el Registro de Defunciones 2023 de Ecuador.

La idea es que este notebook sirva para documentar:

- Cómo se organiza el sistema (topologías).
- Qué rol tiene cada nodo (cliente / servidor / agregador).
- Qué mensajes van a viajar por la red.
- Cómo vamos a serializar y deserializar parámetros del modelo.
- Cómo se sincronizan las rondas globales de entrenamiento.
- Qué estrategias de agregación vamos a comparar.

Aquí solo definimos las decisiones de diseño para luego implementarlas.


## 1. Contexto del problema

Trabajamos con el **Registro de Defunciones 2023** de Ecuador (INEC) para construir un problema de clasificación binaria centrado en un indicador de alto valor en salud pública: **mortalidad prematura por Enfermedades No Transmisibles (ENT / NCD)**, alineado con el estándar oficial **SDG 3.4.1** de Naciones Unidas.

### ¿Qué estamos prediciendo?

Definimos la variable objetivo `is_premature_ncd` según la regla:

| Condición                                                                 | Resultado |
|---------------------------------------------------------------------------|-----------|
| Edad entre **30 y 69 años** **y** causa pertenece a uno de los 4 grupos NCD (cardiovascular, cáncer, diabetes o respiratorias crónicas) | 1 |
| Cualquier otro caso                                                       | 0 |

Para esto:

- Convertimos todas las edades a **años (`edad_anos`)**, combinando la edad reportada con su unidad (`cod_edad`: años, meses, días, horas).
- Usamos `causa103` (nivel intermedio recomendado por INEC) para asignar cada registro a uno de estos grupos:
  - `Cardiovascular`
  - `Cancer`
  - `Diabetes`
  - `Chronic_Respiratory`
  - `No_NCD` (otras causas: infecciosas, perinatales, violentas, accidentes, etc.)

### ¿Por qué este enfoque?

Porque este indicador es **uno de los más vigilados a nivel global**. La OMS lo usa para medir la calidad del sistema de salud, inequidades territoriales y progreso hacia la reducción de muertes evitables.  

Con este problema podemos responder preguntas clave para un país:

- ¿En qué territorios y establecimientos se concentran las muertes prematuras por ENT?
- ¿Qué perfiles demográficos están más expuestos?
- ¿Cómo varían los patrones entre hospitales o provincias sin compartir datos sensibles?

Esto último es **precisamente** donde entra el Aprendizaje Federado.

### ¿Qué hicimos en el notebook anterior (03)?

Ya construimos todo lo necesario para arrancar el sistema federado:

- Pipeline de preprocesamiento (imputación, one-hot, escalado).
- División estratificada train / validación / test.
- Entrenamiento de tres modelos simples:
  - `LogisticRegression`
  - `MLPClassifier`
  - `SGDClassifier`
- Entrenamiento de un **Random Forest** centralizado fuerte.
- Calibración de umbrales para maximizar **F1**, que es la métrica más coherente con el objetivo (detectar la mayor cantidad posible de muertes prematuras sin inflar falsos positivos).

Resultados principales:

- El **Random Forest** fue el mejor modelo centralizado (baseline país completo).
- El **MLPClassifier** fue el mejor modelo simple y será nuestro **modelo base federado**.
- También guardamos una baseline lineal con Regresión Logística.

Modelos guardados:

- `premature_ncd_centralized_best.joblib`  
- `premature_ncd_federated_baseline.joblib`  
- `logreg_premature_ncd_centralized_baseline.joblib`


### ¿Qué resolvemos con este trabajo? (Importancia y aporte)

Este proyecto propone un enfoque **moderno, aplicado y directamente útil para salud pública en Ecuador**:

1. **Modelamos un indicador clave (SDG 3.4.1)** que mide muertes prematuras por ENT, completamente alineado con estándares OMS/ONU.
2. **Aprovechamos el registro nacional de defunciones**, obteniendo un modelo centralizado fuerte que puede ayudar a entender patrones de mortalidad.
3. **Diseñamos una arquitectura de aprendizaje federado**, donde los “hospitales” pueden entrenar modelos colaborativamente **sin compartir datos sensibles**.
4. **Demostramos cómo se comporta el modelo centralizado en cada hospital**, mostrando la heterogeneidad real (datos no-IID), un problema clásico donde el FL tiene ventajas.

**Nuestro aporte es mostrar que Ecuador puede entrenar modelos predictivos de mortalidad prematura por ENT sin centralizar datos sensibles, usando un sistema federado compatible con hospitales reales.**


## 2. Objetivo

Vamos a **documentar la arquitectura** del sistema que vamos a implementar luego en código.

En concreto, dejamos definidos:

1. **Topologías a comparar**
   - Aprendizaje **centralizado clásico** (ya implementado).
   - **Federated Learning centralizado** (un servidor de agregación fijo).
   - **Federated Learning semi-descentralizado** (el rol de agregador rota entre nodos).
   - Boceto de un escenario **totalmente descentralizado** (peer-to-peer).

2. **Nodos y datos**
   - Cómo definimos cada "hospital cliente" a partir del dataset.
   - Por qué los datos quedan **no-IID** (no independientes ni idénticamente distribuidos) entre nodos.

3. **Roles y responsabilidades**
   - Cliente federado.
   - Servidor central (para FL centralizado).
   - Nodo agregador dinámico (para FL semi-descentralizado).

4. **Mensajes y protocolo de comunicación**
   - Qué tipos de mensajes existen (INIT, UPDATE, AGGREGATED, etc.).
   - Qué campos lleva cada mensaje.
   - Cómo serializamos parámetros (JSON con listas anidadas a partir de NumPy).

5. **Rondas globales y sincronización**
   - Cómo se organiza una ronda global de FL.
   - Cuándo se considera que una ronda está "completa".
   - Qué pasa si un nodo no responde.

6. **Estrategias de agregación**
   - **FedAvg ponderado por número de muestras**.
   - **Promedio simple (no ponderado)** como estrategia alternativa.

7. **Métricas para comparar arquitecturas**
   - Accuracy, F1, ROC AUC, tiempos de entrenamiento y costo de comunicación.



## 3. Definición de nodos y escenario no-IID

### 3.1. Cómo definimos los "hospitales clientes"

Nuestro dataset limpio tiene columnas de contexto territorial y de establecimiento, entre ellas:

- `prov_fall`  → provincia de fallecimiento
- `cant_fall`  → cantón de fallecimiento
- `lugar_ocur` → tipo de establecimiento donde ocurrió la muerte

En lugar de contar con un identificador explícito de hospital, definimos un **"hospital cliente"** como una combinación frecuente de:

> `hospital_cliente = (prov_fall, cant_fall, lugar_ocur)`

En el notebook 02 ya se construyó un dataset `defunciones_2023_ncd_hosp_clients.csv` donde:

- Se filtraron las combinaciones más frecuentes (los hospitales "grandes").
- Se seleccionaron **3 hospitales clientes principales**:
  - `Hospital_1`
  - `Hospital_2`
  - `Hospital_3`
- Cada uno tiene miles de registros, con tasas de clase positiva (`is_premature_ncd = 1`) distintas.

Esto nos da un escenario natural para aprendizaje federado:

- Cada *nodo cliente* simula un hospital con su propio subconjunto de defunciones.
- La distribución de características y de la clase no es la misma entre nodos.


In [1]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

hosp_path = DATA_PROCESSED / "defunciones_2023_ncd_hosp_clients.csv"
df_hosp_clients = pd.read_csv(hosp_path)

df_hosp_clients["is_premature_ncd"] = df_hosp_clients["is_premature_ncd"].astype(int)

summary_by_hosp = (
    df_hosp_clients
    .groupby("hospital_cliente")["is_premature_ncd"]
    .agg(["count", "mean"])
    .rename(columns={"count": "n_samples", "mean": "positive_rate"})
    .reset_index()
)

summary_by_hosp

,hospital_cliente,n_samples,positive_rate
0,Hospital_1,5530,0.163472
1,Hospital_2,3003,0.169497
2,Hospital_3,2368,0.139780


La tabla anterior muestra, para cada hospital cliente:

- `n_samples`      → cuántas defunciones aporta ese nodo.
- `positive_rate`  → proporción de muertes prematuras por NCD (**tasa de clase positiva**).

Tenemos algo como esto:

| hospital_cliente | n_samples | positive_rate |
|------------------|-----------|---------------|
| Hospital_1       | 5530      | ≈ 0.16        |
| Hospital_2       | 3003      | ≈ 0.17        |
| Hospital_3       | 2368      | ≈ 0.14        |

Esto confirma que:

- Cada nodo tiene un **tamaño de muestra distinto** (desbalance en cantidad de datos).
- Las **tasas de clase positiva son distintas**, es decir, los nodos no ven la misma proporción de muertes prematuras por NCD.

Es un escenario claramente **no-IID**, alineado con lo que se observa en la práctica cuando se trabaja con hospitales reales.


## 4. Topologías del sistema a comparar

En este proyecto vamos a comparar explícitamente tres configuraciones:

1. **Aprendizaje centralizado clásico** (baseline).
2. **Aprendizaje federado centralizado** (un servidor de agregación fijo).
3. **Aprendizaje federado semi-descentralizado** (el agregador rota entre nodos).

Y dejaremos un bosquejo para un posible **FL totalmente descentralizado** (peer-to-peer).

A continuación describimos cada topología.

### 4.1. Aprendizaje centralizado clásico (baseline)

Este es el escenario que ya implementamos en el notebook 03:

- Todo el dataset limpio 2023 (o el subconjunto elegido) se carga en un **solo servidor**.
- Entrenamos un modelo centralizado (Random Forest, en nuestro caso) con acceso a **todas las muestras**.
- Evaluamos el desempeño en un conjunto de test estratificado.

En términos de arquitectura:

- No hay comunicación entre nodos.
- No hay privacidad por diseño: todos los registros se centralizan.

Este escenario sirve como **upper bound** de desempeño: nos dice qué tan bien podemos hacerlo cuando no hay restricciones de privacidad ni de reparto de datos.


### 4.2. Federated Learning centralizado (un servidor fijo)

En el escenario de **FL centralizado**:

- Tenemos un **servidor de agregación** fijo (puede estar en la misma máquina donde corrimos el modelo centralizado, pero lógicamente es otro rol).
- Cada **nodo cliente** (Hospital_1, Hospital_2, Hospital_3) mantiene localmente su fragmento de datos.
- El entrenamiento ocurre en **rondas globales**. En cada ronda:

  1. El servidor envía a cada cliente el **modelo global actual** (sus parámetros).
  2. Cada cliente:
     - Carga los parámetros recibidos.
     - Entrena localmente durante `E` épocas sobre sus datos.
     - Envía de vuelta al servidor sus **parámetros actualizados** y el número de muestras usadas (`n_samples`).
  3. El servidor espera a recibir las actualizaciones de todos (o de una fracción) de los clientes.
  4. El servidor aplica una **regla de agregación** (por ejemplo, FedAvg) sobre los parámetros recibidos.
  5. El servidor actualiza el modelo global y repite el proceso en la siguiente ronda.

En esta topología, el servidor es:

- El **único punto de agregación**.
- La **única fuente de verdad** del modelo global.
- Responsable de registrar métricas por ronda y de coordinar la sincronización.


### 4.3. Federated Learning semi-descentralizado (agregador rotativo)

En el escenario **semi-descentralizado**:

- Eliminamos el **servidor fijo** como componente lógico.
- En cada ronda elegimos **uno de los nodos clientes** para que actúe como **agregador temporal**.
- La elección del agregador puede seguir distintas políticas:
  - **Round-robin**: en la ronda 1 el agregador es el Hospital_1, en la 2 el Hospital_2, en la 3 el Hospital_3, y se repite.
  - **Mejor desempeño local**: elegimos como agregador el nodo que haya obtenido mejor F1 en la ronda anterior.
  - **Selección aleatoria**, con probabilidad uniforme o ponderada.

Para este proyecto vamos a usar una política **simple y reproducible**:

> **Round-robin entre los 3 hospitales clientes.**

Flujo por ronda:

1. Elegimos un **agregador actual** `A_t` (por ejemplo, Hospital_2).
2. `A_t` envía el modelo global de referencia a los demás nodos.
3. Todos los nodos (incluyendo `A_t`) entrenan localmente durante `E` épocas.
4. Los nodos envían sus parámetros actualizados a `A_t` junto con `n_samples`.
5. `A_t` agrega los parámetros (por ejemplo, usando FedAvg).
6. `A_t` difunde el modelo global actualizado a todos los nodos.


Observaciones:

- Lógicamente el sistema sigue siendo **centralizado por ronda**, pero el rol de agregador **se mueve** entre nodos.
- Esto permite estudiar un escenario más cercano a una red **entre pares** donde no hay un servidor fijo privilegiado.


### 4.4. Boceto de un FL completamente descentralizado (peer-to-peer)

Como extensión conceptual, podemos mencionar una topología **peer-to-peer** donde:

- No existe ningún servidor ni agregador designado.
- Cada nodo intercambia parámetros directamente con algunos vecinos según una **topología de red** (por ejemplo, un anillo o un grafo aleatorio).
- El modelo se va mezclando a través de rondas de comunicación local (algoritmos tipo gossip).

En este proyecto **no vamos a implementar** esta variante, pero podemos plantearla como futuro trabajo:

- Nuestro modelo base MLP se podría entrenar en un **anillo de hospitales**, donde cada hospital promedia su modelo con el de su vecino en cada ronda.
- El paper puede incluir un pequeño esquema conceptual y discutir los retos: convergencia más lenta, necesidad de múltiples pasos de gossip, etc.


## 5. Roles en el sistema: cliente, servidor y agregador

### 5.1. Cliente federado (hospital)

Cada **cliente federado** representa un hospital (o grupo de hospitales) con su propio subconjunto de datos:

- Tiene acceso **únicamente** a sus registros de defunciones.
- Nunca envía datos crudos (ni variables sensibles) a otros nodos.
- Solo comparte **parámetros del modelo** y metadatos agregados.

Responsabilidades principales:

1. **Cargar el modelo base federado** (MLP) y el pipeline de preprocesamiento desde el modelo `premature_ncd_federated_baseline.joblib`.
2. **Recibir parámetros iniciales** del modelo global.
3. **Reemplazar** los parámetros de su modelo local con los parámetros globales recibidos.
4. Entrenar localmente durante `E` épocas con sus datos (`X_h`, `y_h`).
5. **Medir métricas locales** (F1, accuracy, etc.) en un conjunto de validación local.
6. **Enviar al agregador**:
   - Sus parámetros actualizados.
   - `n_samples` usados en el entrenamiento.
   - Opcionalmente, métricas locales para monitoreo.
7. Recibir el nuevo modelo global y repetir el proceso en la siguiente ronda.


### 5.2. Servidor central (solo en FL centralizado)

El **servidor central** aparece únicamente en la topología de FL centralizado.

Responsabilidades:

- Mantener el **modelo global** (mismas dimensiones y arquitectura que el modelo base federado MLP).
- Coordinar las **rondas globales**:
  - Difundir parámetros a todos los clientes.
  - Esperar actualizaciones.
  - Agregar parámetros.
  - Actualizar el modelo global.
- Registrar en logs:
  - Métricas globales por ronda (por ejemplo, F1 promedio estimado sobre un conjunto central de validación o combinando métricas locales).
  - Número de mensajes y bytes enviados/recibidos (costo de comunicación).
- Manejar casos de fallo:
  - Si un cliente no responde en una ronda, se ignora su actualización para esa ronda y se continúa con los demás.


### 5.3. Nodo agregador dinámico (en FL semi-descentralizado)

En la topología semi-descentralizada:

- No hay un servidor dedicado.
- Uno de los clientes asume el rol de **agregador** en una ronda dada.

Responsabilidades del agregador en la ronda `t`:

1. Iniciar la ronda difundiendo el modelo global local que posee.
2. Recibir parámetros de los otros nodos.
3. Aplicar la función de **agregación** (FedAvg ponderado, o la estrategia alternativa).
4. Actualizar su copia del modelo global.
5. Difundir el modelo global actualizado a todos los clientes.

En la práctica, cada nodo cliente deberá tener la lógica necesaria para:

- Actuar como **cliente normal** en las rondas donde no le toca agregar.
- Actuar como **agregador** cuando es seleccionado por la política de rotación (round-robin).


## 6. Mensajes y protocolo de comunicación

A nivel de implementación, vamos a usar **sockets TCP** y **JSON** como formato de serialización.

- TCP nos da un canal confiable (sin pérdida de mensajes).
- JSON es fácil de depurar y suficiente para el tamaño de modelos que manejamos (MLP pequeño).

### 6.1. Tipos de mensajes

Definimos los siguientes tipos lógicos de mensaje:

1. `INIT_CONFIG`
   - Se envía al inicio del experimento.
   - Permite compartir:
     - IDs de los nodos.
     - Número total de rondas.
     - Hiperparámetros globales (epochs locales, batch size, etc.).

2. `GLOBAL_MODEL`
   - El servidor (o agregador) envía el **modelo global actual** a los clientes.
   - Contiene:
     - `round_id`
     - `model_name` (por ejemplo, `"mlp_federated_baseline"`)
     - `weights` y `biases` serializados.

3. `LOCAL_UPDATE`
   - El cliente envía su actualización local tras entrenar.
   - Contiene:
     - `round_id`
     - `client_id`
     - `n_samples`
     - `weights` y `biases` actualizados.
     - Métricas locales opcionales (`f1_local`, `loss_local`, etc.).

4. `AGGREGATED_MODEL`
   - El servidor (o agregador) difunde el **modelo agregado** tras combinar las actualizaciones.
   - Estructura similar a `GLOBAL_MODEL`.

5. `HEARTBEAT` / `ACK`
   - Mensajes ligeros para confirmar que los nodos están vivos y que recibieron correctamente un paquete importante.


### 6.2. Formato de los parámetros del modelo (MLP federado)

El modelo base federado que vamos a usar es equivalente al `MLPClassifier` central ya entrenado:

- Una capa oculta de tamaño moderado (por ejemplo `(32,)`).
- Activación ReLU.
- Capa de salida con una neurona (clasificación binaria).

En scikit-learn, los parámetros del MLP se almacenan como:

- `coefs_`   → lista de matrices de pesos por capa.
- `intercepts_` → lista de vectores de sesgos por capa.

Para poder enviarlos por JSON, haremos:

1. Convertir cada matriz/ vector en una **lista anidada de floats** (con `.tolist()`).
2. Enviar también la **forma** (`shape`) de cada matriz para reconstruirla fácilmente.

Un ejemplo de estructura JSON para los parámetros sería:

```json
{
  "round_id": 3,
  "model_name": "mlp_federated_baseline",
  "weights": [
    {
      "name": "W1",
      "shape": [n_features, 32],
      "values": [[...], [...], ...]
    },
    {
      "name": "b1",
      "shape": [32],
      "values": [...]
    },
    {
      "name": "W2",
      "shape": [32, 1],
      "values": [[...], ...]
    },
    {
      "name": "b2",
      "shape": [1],
      "values": [...]
    }
  ]
}
```

A nivel de implementación, tendremos funciones auxiliares para:

- **Extraer parámetros** del modelo MLP de scikit-learn y convertirlos a un diccionario serializable.
- **Reconstruir parámetros** a partir del diccionario recibido y asignarlos al modelo local.


In [2]:
# Esqueleto (aún sin implementación concreta) de cómo se verían las funciones auxiliares
# para extraer y cargar parámetros del modelo MLP federado.

def extract_mlp_parameters(mlp_model):
    """Convierte coefs_ e intercepts_ de un MLP de scikit-learn a un dict serializable por JSON."""
    params = []
    for i, (w, b) in enumerate(zip(mlp_model.coefs_, mlp_model.intercepts_)):
        params.append({
            "name": f"W{i+1}",
            "shape": list(w.shape),
            "values": w.tolist(),
        })
        params.append({
            "name": f"b{i+1}",
            "shape": list(b.shape),
            "values": b.tolist(),
        })
    return params


def load_mlp_parameters(mlp_model, params):
    """Carga parámetros en un MLP de scikit-learn a partir de un dict serializable."""
    import numpy as np

    coefs = []
    intercepts = []
    # Asumimos que params viene en orden [W1, b1, W2, b2, ...]
    for i in range(0, len(params), 2):
        W_info = params[i]
        b_info = params[i + 1]
        coefs.append(np.array(W_info["values"], dtype=float))
        intercepts.append(np.array(b_info["values"], dtype=float))

    mlp_model.coefs_ = coefs
    mlp_model.intercepts_ = intercepts
    return mlp_model

Estas funciones **no se usan todavía** en este notebook, pero dejan claro el tipo de serialización que se implementará después en los scripts de servidor y clientes.


## 7. Rondas globales y sincronización

El entrenamiento federado se organiza en **rondas globales** numeradas `t = 1, 2, ..., T`.

### 7.1. Parámetros de alto nivel (configuración global)

Definimos algunos hiperparámetros que se mantendrán consistentes entre las implementaciones:

- `N_CLIENTS`      → número de nodos clientes (en nuestro caso, 3 hospitales).
- `GLOBAL_ROUNDS`  → número total de rondas globales `T`.
- `LOCAL_EPOCHS`   → número de épocas de entrenamiento local en cada ronda (por ejemplo, 1–5).
- `BATCH_SIZE`     → tamaño de mini-batch en el entrenamiento local.
- `AGG_STRATEGY`   → estrategia de agregación (`"fedavg_weighted"` o `"fedavg_uniform"`).
- `AGG_POLICY`     → política de selección de agregador en el caso semi-descentralizado (`"fixed_server"` o `"round_robin"`).

Guardamos un pequeño diccionario de configuración que luego podremos reutilizar:



In [3]:
GLOBAL_CONFIG = {
    "N_CLIENTS": 3,
    "CLIENT_IDS": ["Hospital_1", "Hospital_2", "Hospital_3"],
    "GLOBAL_ROUNDS": 10,
    "LOCAL_EPOCHS": 3,
    "BATCH_SIZE": 64,
    "AGG_STRATEGY": "fedavg_weighted",  # alternativa: "fedavg_uniform"
    "AGG_POLICY": "fixed_server",       # alternativa: "round_robin" (semi-descentralizado)
}

GLOBAL_CONFIG

{'N_CLIENTS': 3,
 'CLIENT_IDS': ['Hospital_1', 'Hospital_2', 'Hospital_3'],
 'GLOBAL_ROUNDS': 10,
 'LOCAL_EPOCHS': 3,
 'BATCH_SIZE': 64,
 'AGG_STRATEGY': 'fedavg_weighted',
 'AGG_POLICY': 'fixed_server'}

Este diccionario es solo una plantilla. En la implementación real lo ajustaremos según los experimentos.

### 7.2. Ciclo de una ronda global en FL centralizado

En el caso de **FL centralizado**, una ronda global `t` se ve así:

1. **Broadcast del modelo global**
   - El servidor envía el modelo global actual a todos los clientes en un mensaje `GLOBAL_MODEL`.

2. **Entrenamiento local**
   - Cada cliente:
     - Reconstruye el modelo local a partir de los parámetros recibidos.
     - Entrena localmente `LOCAL_EPOCHS` épocas con sus datos.
     - Calcula métricas locales (opcional).

3. **Envío de actualizaciones**
   - Cada cliente envía un mensaje `LOCAL_UPDATE` al servidor con:
     - `round_id = t`
     - `client_id`
     - `n_samples`
     - `params` (lista de pesos y sesgos serializados)
     - Métricas locales opcionales.

4. **Agregación**
   - El servidor espera actualizaciones de todos los clientes (o de una fracción mínima configurada).
   - Aplica la función de agregación seleccionada (`AGG_STRATEGY`).
   - Actualiza el modelo global.

5. **Registro de resultados**
   - El servidor registra métricas globales para la ronda `t`.
   - Opcionalmente, evalúa el modelo global en un subconjunto de validación central.

Luego pasa a la ronda `t+1` hasta completar `GLOBAL_ROUNDS`.


### 7.3. Ciclo de una ronda global en FL semi-descentralizado

En el **FL semi-descentralizado** con política `round_robin`:

- El conjunto de clientes es el mismo (`Hospital_1`, `Hospital_2`, `Hospital_3`).
- En cada ronda `t` elegimos un **agregador** distinto, por ejemplo:

  - Ronda 1 → `Hospital_1`
  - Ronda 2 → `Hospital_2`
  - Ronda 3 → `Hospital_3`
  - Ronda 4 → `Hospital_1` (y así sucesivamente)

El ciclo de la ronda `t` es:

1. El agregador `A_t` difunde el modelo global actual (que guarda localmente) en un mensaje `GLOBAL_MODEL`.
2. Todos los nodos (incluyendo `A_t`) entrenan localmente y generan sus `LOCAL_UPDATE`.
3. Los clientes envían sus actualizaciones de vuelta a `A_t`.
4. `A_t` aplica la estrategia de agregación (`AGG_STRATEGY`) y actualiza su modelo global.
5. `A_t` difunde el modelo global actualizado a los demás nodos.

En la práctica, toda la lógica de agregación que antes estaba en el servidor se moverá al nodo que toque según `AGG_POLICY = "round_robin"`.


## 8. Estrategias de agregación a comparar

La rúbrica del proyecto pide **comparar al menos dos estrategias de agregación**.

En este trabajo vamos a comparar:

1. **FedAvg ponderado por número de muestras** (`fedavg_weighted`)
2. **Promedio simple no ponderado** (`fedavg_uniform`)

### 8.1. FedAvg ponderado (`fedavg_weighted`)

Sea \(K\) el número de clientes participantes en una ronda, y \(w_k\) los parámetros del modelo del cliente \(k\) después del entrenamiento local, con \(n_k\) muestras usadas.

La actualización global es:

\[
w^{(t+1)} = \frac{\sum_{k=1}^K n_k \, w_k}{\sum_{k=1}^K n_k}
\]

Es decir, cada cliente pesa proporcionalmente a la cantidad de datos que aporta.

Ventajas:

- Se parece a entrenar con todos los datos mezclados: los nodos grandes tienen más peso en la actualización.
- Suele maximizar el desempeño global cuando el objetivo principal es minimizar la pérdida promedio sobre todos los registros.

Desventajas:

- Los nodos pequeños pueden quedar subrepresentados (menor influencia en el modelo global).

### 8.2. Promedio simple (`fedavg_uniform`)

En la variante de promedio simple no ponderado, la actualización es:

\[
w^{(t+1)} = \frac{1}{K} \sum_{k=1}^K w_k
\]

Todos los clientes tienen el mismo peso, independientemente del tamaño de su conjunto de datos.

Ventajas:

- Más sencillo conceptualmente.
- Da más peso relativo a los nodos pequeños (útil si queremos cierta noción de equidad entre hospitales).

Desventajas:

- Desde el punto de vista de la pérdida global, puede ser subóptimo si los tamaños de los nodos difieren mucho.

En el paper podremos comparar ambos enfoques en términos de:

- F1 global en test (comparado contra el modelo centralizado).
- Variación de métricas por hospital.
- Costo de comunicación (igual en ambos casos, pero podemos comentar si la convergencia difiere).

In [4]:
def aggregate_fedavg_weighted(client_updates):
    """Esqueleto de agregación FedAvg ponderado.
    client_updates: lista de dicts con claves:
        - 'n_samples'
        - 'params'  (lista de pesos y biases)
    Aquí solo dejamos la firma; la implementación real irá en los scripts de FL.
    """
    raise NotImplementedError


def aggregate_fedavg_uniform(client_updates):
    """Esqueleto de agregación usando promedio simple (no ponderado)."""
    raise NotImplementedError

Estas firmas dejan claro que:

- Recibiremos una lista de actualizaciones, una por cliente.
- Cada entrada tendrá al menos `n_samples` y la lista de parámetros serializados.
- Tendremos dos funciones separadas para cambiar fácilmente de estrategia en los experimentos.


## 9. Métricas y logging para comparar arquitecturas

Para que la comparación entre arquitecturas sea consistente, usaremos el mismo conjunto de métricas que en el escenario centralizado:

- **Accuracy**
- **Precision**
- **Recall**
- **F1-score** (métrica principal)
- **ROC AUC**
- Matrices de confusión y curvas ROC / Precision-Recall (en el notebook de resultados).

Además, para el caso federado vamos a registrar explícitamente:

- **Tiempos de entrenamiento local** por nodo y por ronda.
- **Tiempo de agregación** por ronda.
- **Tamaño de los mensajes** (al menos aproximado, en bytes o kB).
- **Número total de mensajes** intercambiados.

En la práctica, tendremos una estructura de logging tipo:

```python
history = {
    "round": [],
    "arch": [],           # 'centralized_fl' o 'semi_decentralized_fl'
    "agg_strategy": [],   # 'fedavg_weighted' o 'fedavg_uniform'
    "global_f1_val": [],
    "global_f1_test": [],
    "total_comm_bytes": [],
    "train_time_sec": [],
    # etc.
}
```

Esta información alimentará tanto:

- Los **gráficos** de comparación (accuracy / F1 vs. ronda, pérdida vs. ronda).
- Como las **tablas** del paper (centralizado vs FL centralizado vs FL semi-descentralizado).
